In [26]:
#windows machine
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

In [27]:
#!pip install transformers pandas numpy

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [22]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [28]:
#Test the model functionality

#tokenize the sentence
tokens = tokenizer.encode('This is the best streak I have ever had.', return_tensors='pt')
print(tokens)

tensor([[  101, 10372, 10127, 10103, 11146, 69217,   151, 10574, 15765, 10407,
           119,   102]])


In [29]:
#results on a scale of 1-5 with probability displayed respectively
result = model(tokens)
print(result.logits)

tensor([[-1.8257, -2.1205, -1.2404,  0.6951,  3.8157]],
       grad_fn=<AddmmBackward0>)


In [30]:
#Result of sentiment analysis on a scale of 1-5, 1 being extremely negative, 5 being extremely positive
int(torch.argmax(result.logits))+1

5

In [31]:
#Lets grab some DATA!!!

#!pip install pathlib
#!pip install praw==7.5.0

In [ ]:
from pathlib import Path  
import praw
from praw.models import MoreComments
import pandas as pd
filepath = Path("C:/Users/Charlie Lu\Desktop/Sentiment Analysis/RedditAPI/test.csv")  #Location of output CSV